### Fair Readers and Writers with Semaphores (Java) [8 points]

Implement a fair solution in Java with readers and writers following the synchronization scheme of the course notes. In the template below, there is only a single reader and a single writer. The reader busily keeps reading and computes the maximum of all numbers read. The writer generates in 1-second intervals numbers `0 .. numIters` starting from the middle one and wrapping around, i.e. generating the maximum after about half the iterations. The variable `numIters` is the command line parameter. The synchronization scheme has to ensure that the writer is given preference, so if there is more than one reader, all readers can get updates from the writer. Your solution must be correct for multiple readers and writers, even if the code below creates only one reader and a single writer.

By default, threads in Java are not created as _daemon threads,_ meaning that they will continue after the main thread terminates. Below, the main program waits only for the writer to terminate, and the reader thread is made a daemon thread.

Be careful to declare only those variables `volatile` that need to be declared volatile. Otherwise, the program may run slower unnecessarily. Play around to see what happens if you don't declare variables `volatile`!

In [16]:
%%writefile ReaderWriter.java
import java.util.concurrent.Semaphore;
import java.lang.String;

public class ReaderWriter {
    /* global variables and semaphores */
    public static Semaphore e = new Semaphore(1);
    public static Semaphore r = new Semaphore(0);
    public static Semaphore w = new Semaphore(0);
    public static int nr = 0;
    public static int nw = 0;
    public static int dr = 0;
    public static int dw = 0;
    public static volatile int data = 0;
    public static int numIters = 0;
    public static volatile int maxdata;

    static void reader() {
        maxdata = 0;
        System.out.println("Reader starting");
        while (true) {
            /* entry protocol */
            try{e.acquire();} catch (Exception e) {}
            if (nw > 0 || dw > 0) {
                dr += 1;
                e.release();
                try{r.acquire(); } catch (Exception e) {}
            }
            nr += 1;
            if (dr > 0){
                dr -=1;
                r.release();
            }
            else{
                e.release();
            }
            if (data > maxdata) System.out.printf("Reader in critical section read %d\n", data);
            maxdata = data > maxdata ? data : maxdata;
            /* exit protocol */     
            try{e.acquire();} catch (Exception e) {}
            nr -= 1;
            if (nr == 0 && dw > 0){
                dw-=1;
                w.release();
            }
            else{
                e.release();
            }
        }
    }

    static void writer() {
        System.out.printf("Writer starting\n");
        for (int i = 0; i < numIters; i++) {
            /* entry protocol */
            try{e.acquire();} catch (Exception e) {}
            System.out.println("Writer trying to enter");
            if (nr > 0 || nw > 0){
                dw +=1;
                e.release();
                try{w.acquire();} catch (Exception e) {}
            }
            nw += 1;
            e.release();
                
            System.out.printf("Writer in critical section\n");
            data = (i + numIters / 2) % numIters;
            System.out.printf("Writer writing %d\n", data);
            /* exit protocol */
            try{e.acquire();} catch (Exception e) {}
            nw -= 1;
            if (dw > 0){
                dw -=1;
                w.release();
            }
            else if (dr > 0){
                dr -= 1;
                r.release();
            }
            else {
                e.release();
            }
            try {Thread.sleep(1000);} catch (Exception e) {}; // sleep 1 sec
        }
    }
    public static void main(String args[]) {
        numIters = Integer.parseInt(args[0]);
        Thread r = new Thread() {{setDaemon(true);} public void run() {reader();}};
        Thread w = new Thread() {public void run() {writer();}};

        r.start(); w.start();
        try {w.join();} catch (Exception e) {}
        System.out.printf("Max data %d\n", maxdata);       
    }
}

Overwriting ReaderWriter.java


In [17]:
!javac ReaderWriter.java

In [18]:
!java ReaderWriter 5

Writer starting
Reader starting
Writer trying to enter
Writer in critical section
Writer writing 2
Reader in critical section read 2
Writer trying to enter
Writer in critical section
Writer writing 3
Reader in critical section read 3
Writer trying to enter
Writer in critical section
Writer writing 4
Reader in critical section read 4
Writer trying to enter
Writer in critical section
Writer writing 0
Writer trying to enter
Writer in critical section
Writer writing 1
Max data 4
